In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# --- 1. Load and Prepare Data ---
print("Loading and preparing data...")
try:
    df = pd.read_csv("../data/final_model_dataset_v2.csv")
except Exception as e:
    print(f"An error occurred loading the file: {e}")
    # Stop if file not found
    exit()

ignore_cols = ["cation", "cation_ion_name", "anion", "anion_ion_name"]
target_col = "CO2_solubility"
feature_cols = [col for col in df.columns if col not in ignore_cols + [target_col]]

X = df[feature_cols]
y = df[target_col].values

# Train-validation split (using the same random_state=42 for a fair comparison)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 2. Feature Scaling ---
# Note: Random Forest is not sensitive to feature scaling, but we
# scale the data anyway to keep the workflow identical to the Neural Network
# for a perfectly fair comparison.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
print("Data preparation complete.")

# --- 3. Initialize and Train the Random Forest Model ---
print("\n--- Training Random Forest Model ---")
# n_estimators=100 means the model will build 100 "decision trees"
# n_jobs=-1 will use all your computer's CPU cores to speed up training
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

rf_model.fit(X_train_scaled, y_train)
print("--- Training Complete ---")

# --- 4. Evaluate the Model ---
print("\n--- Model Evaluation ---")
y_pred_rf = rf_model.predict(X_val_scaled)

r2_rf = r2_score(y_val, y_pred_rf)
mae_rf = mean_absolute_error(y_val, y_pred_rf)

print(f"Random Forest R² score: {r2_rf:.4f}")
print(f"Random Forest MAE score: {mae_rf:.4f}")

# --- 5. Visualize the Results (Parity Plot) ---
plt.figure(figsize=(8, 8))
plt.scatter(y_val, y_pred_rf, alpha=0.7, label='Predictions')
plt.xlabel('Experimental CO2 Solubility (Actual)')
plt.ylabel('Predicted CO2 Solubility (Model)')
plt.title('Random Forest Performance: Predicted vs. Actual')

# Add R² score to the plot
plt.text(0.05, 0.9, f'$R^2 = {r2_rf:.4f}$', transform=plt.gca().transAxes, fontsize=14,
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Add a 45-degree line for reference
lims = [min(plt.xlim()[0], plt.ylim()[0]), max(plt.xlim()[1], plt.ylim()[1])]
plt.plot(lims, lims, 'r--', alpha=0.75, zorder=0, label='Perfect Prediction')
plt.xlim(lims)
plt.ylim(lims)
plt.grid(True)
plt.legend()
plt.show()

# --- 6. Feature Importance Plot ---
print("\n--- Feature Importance Analysis ---")
importances = rf_model.feature_importances_
feature_names = X.columns.tolist()

# Create a DataFrame for easy sorting and plotting
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
# Sort by importance and get the top 20 features
top_20_features = importance_df.sort_values(by='Importance', ascending=False).head(20)

# Plot the top 20 features
plt.figure(figsize=(10, 10))
plt.barh(top_20_features['Feature'], top_20_features['Importance'])
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.title("Top 20 Most Important Features (Random Forest)")
plt.gca().invert_yaxis() # Display the most important feature at the top
plt.tight_layout()
plt.show()